In [3]:
import requests
import json
import pprint
from aiohttp import ClientSession, FormData
import base58

In [2]:
>>> import hashlib
>>> import multihash
>>> from multibase import encode, decode
>>> m = hashlib.sha256()
>>> m.update(b"Hello World!")
>>> multi = multihash.encode(m.digest(), 'sha2-256')
>>> mem = encode('base58btc', multi)
print(mem)
b'zQmWvQxTqbG2Z9HPJgG57jjwR154cKhbtJenbyYTWkjgF3e' 
print(type(mem))

decode_base = decode(mem)
decode_hash = multihash.decode(decode_base)
hashed = decode_hash[3]
algo = decode_hash[1]
print(decode_hash[1])

b'zQmWvQxTqbG2Z9HPJgG57jjwR154cKhbtJenbyYTWkjgF3e'
<class 'bytes'>
sha2-256


In [8]:
id(m)

139743949484224

In [20]:
async with ClientSession() as session:
            result = await session.post("https://data-vault.eu/oauth/token", json={
                "client_id": "8SX1-RBazhic9ftG4HyG2XegrQ2kdLGu0hd-Ty3IZnE", 
                "client_secret": "gEg9_tTh2V1ZK6piL9tZ38YO-1xqXLDXCccJS4uQUJg",
                "grant_type": "client_credentials"
            })
            print(type(result.status))
            result = await result.text()
            token = json.loads(result)
            print(result)

<class 'int'>
{"access_token":"hiHunSAlxwSiF_MIsCeKJ4oJaFcN3x2iXRLTiha4DcA","token_type":"Bearer","expires_in":7200,"created_at":1606907441,"username":"Karol"}


In [7]:
async with ClientSession() as session:
    result = await session.post("https://data-vault.eu/api/repos/dip.data/items", json={
        "test": "data"
    }, headers={
        "Authorization": "Bearer " + token['access_token']
    })

    result = await result.text()
    save = json.loads(result)
    print(result)

{"id":1716575}


In [25]:
async with ClientSession() as session:
    result = await session.get("https://data-vault.eu/api/meta/usage", headers={
        "Authorization": "Bearer " + token['access_token']
    })
    
    result = await result.text()
    print(result)


ClientConnectorError: Cannot connect to host data-vault.eu:443 ssl:default [Connect call failed ('45.157.176.15', 443)]

In [8]:
url = "https://data-vault.eu/api/items/"+str(save["id"])+"/details"
print(url)
async with ClientSession() as session:
            result = await session.get(url, headers={
                "Authorization": "Bearer " + token['access_token']
            })
            result = await result.text()
            load = json.loads(result)
            print(result)

https://data-vault.eu/api/items/1716575/details
{"id":1716575,"repo_id":5785,"merkle_id":null,"value":"{\"test\":\"data\",\"id\":1716575}","oyd_hash":null,"created_at":"2020-12-02T10:54:23.640Z","updated_at":"2020-12-02T10:54:23.676Z","oyd_source_pile_id":null,"dri":null,"schema_dri":null,"mime_type":null,"repo_name":"dip.data","access_count":18}


In [9]:
url = "https://data-vault.eu/api/repos/dip.data/items/" + str(load['id'])
async with ClientSession() as session:
            result = await session.put(url, json={
                "change": "data"
            }, headers={
                "Authorization": "Bearer " + token['access_token']
            })
            
            result = await result.text()
            update = json.loads(result)
            print(result)

{"item_id":1716575}


In [10]:
url = "https://data-vault.eu/api/items/"+str(update['item_id'])+"/details"
print(url)
async with ClientSession() as session:
            result = await session.get(url, headers={
                "Authorization": "Bearer " + token['access_token']
            })
            result = await result.text()
            load = json.loads(result)
            print(result)

dri_value = 262462624234
dri_schema_value = 1616442423

https://data-vault.eu/api/items/1716575/details
{"id":1716575,"repo_id":5785,"merkle_id":null,"value":"{\"change\":\"data\",\"id\":\"1716575\"}","oyd_hash":null,"created_at":"2020-12-02T10:54:23.640Z","updated_at":"2020-12-02T10:54:46.904Z","oyd_source_pile_id":null,"dri":null,"schema_dri":null,"mime_type":null,"repo_name":"dip.data","access_count":20}


In [12]:
url = "https://data-vault.eu/api/data"
print(url)
async with ClientSession() as session:
            result = await session.post(url, headers={
                "Authorization": "Bearer " + token['access_token']
            }, json={
              "content": {"abs": "test asdasd"},
              "dri": dri_value,
              "schema_dri": dri_schema_value,
              "mime_type": "application/json",
              "table_name": "dip.dataa"
            })
            result = await result.text()
            load = json.loads(result)
            print(result)

https://data-vault.eu/api/data
{"processed":1,"responses":[{"id":1625181,"status":200}]}


In [13]:
url = f"https://data-vault.eu/api/data?dri={dri_value}&schema_dri={dri_schema_value}"
async with ClientSession() as session:
            result = await session.get(url, headers={
                "Authorization": "Bearer " + token['access_token']
            })
            result = await result.text()
            load = json.loads(result)
            print(result)

{"data":[{"content":{"content":{"abs":"test asdasd"}},"id":1625181,"dri":"262462624234","schema_dri":"1616442423"}],"usage-policy":"\u003chttp://w3id.org/semcon/ns/ontology#ContainerPolicy\u003e a \u003chttp://www.w3.org/2002/07/owl#Class\u003e;\n    \u003chttp://www.w3.org/2002/07/owl#equivalentClass\u003e [\n    a \u003chttp://www.w3.org/2002/07/owl#Class\u003e;\n    \u003chttp://www.w3.org/2002/07/owl#intersectionOf\u003e ([\n    a \u003chttp://www.w3.org/2002/07/owl#Restriction\u003e;\n    \u003chttp://www.w3.org/2002/07/owl#onProperty\u003e \u003chttp://www.specialprivacy.eu/langs/usage-policy#hasData\u003e;\n    \u003chttp://www.w3.org/2002/07/owl#someValuesFrom\u003e \u003chttp://www.specialprivacy.eu/langs/usage-policy#AnyData\u003e\n    ] [\n    a \u003chttp://www.w3.org/2002/07/owl#Restriction\u003e;\n    \u003chttp://www.w3.org/2002/07/owl#onProperty\u003e \u003chttp://www.specialprivacy.eu/langs/usage-policy#hasProcessing\u003e;\n    \u003chttp://www.w3.org/2002/07/owl#some

In [6]:

            



            
            

            



            
            

            

            
# url = "https://data-vault.eu/api/repos/dip.data/items"
# async with ClientSession() as session:
#     result = await session.get(url, headers={"Authorization": "Bearer " + token['access_token']})
#     result = await result.text()
#     update = json.loads(result)
#     print(result)            
            
            


<class 'int'>
{"access_token":"Je1IQTiam_K-6-9KzTus38xWUR2TViHpa4t3IMBDmmU","token_type":"Bearer","expires_in":7200,"created_at":1606899999,"username":"Karol"}
{"id":1716489}


In [5]:
async with ClientSession() as session:
            result = await session.get("https://data-vault.eu/api/items/1610004/details", headers={
                "Authorization": "Bearer " + token['access_token']
            })
            result = await result.text()
            load = json.loads(result)
            print(result)

{"id":1610004,"repo_id":5785,"merkle_id":null,"value":"{\"test\":\"data\",\"id\":1610004}","oyd_hash":null,"created_at":"2020-10-02T12:53:05.392Z","updated_at":"2020-10-02T12:53:05.418Z","oyd_source_pile_id":null,"dri":null,"schema_dri":null,"mime_type":null,"repo_name":"dip.data","access_count":18}


In [4]:

url = 'http://data-vault.localhost/api/v1/files'
data = FormData()
data.add_field('file',
               "memes",
               filename='abba',
               content_type='application/json')

async with ClientSession() as session:
    resp = await session.post(url=url, data=data)
    print(await resp.text())

{"record_dri":"zQmRBXgMkjh1Vw9MB3XZt3cHDRxmX15b7xmHLsVFcjZv8th","content_dri":"zQmNmDadhKMDtF1C8nJBwxzync5RytDBJruo9rQ2mgKnQMj"}


In [8]:
add = requests.post('http://data-vault.localhost/api/v1/files', files = {
    'file': ('test_file', 'test_test_yes')
})
print(add)
print(add.text)
ids = json.loads(add.text)

<Response [200]>
{"record_dri":"zQmcXWdhiJSaWg25mXKPTZAp4EAt7SjfEPAjKdjSDUXfDXn","content_dri":"zQmVAxg5Ncz6C8zjx4bab7uQB2W8AGqkAVcRAi2VceQN6Cc"}


In [9]:
get = requests.get('http://data-vault.localhost/api/v1/files/' + ids['content_dri'])
print(get.text)

test_test_yes


In [50]:
get = requests.get('http://data-vault.localhost/api/v1/files')
print(get.text)
memes = json.loads(get.text)
len(memes)
pprint.pprint(memes)

[{"DRI":"zQmP9LkvbGXK2aaVuitR8WCZQYeKosFvcFQQjunZkz8NZ4F","file":{"name":"zQmbB6LSS92nZEmEKZLQH6bzd9EeBXuW42DuWpSmD76aMQX","type":"json","contentDRI":"zQmbB6LSS92nZEmEKZLQH6bzd9EeBXuW42DuWpSmD76aMQX"}},{"DRI":"zQmVHm8oRsiRgPv5wUQ1c286cLj9Z3iBsd4ah2z3bMGExbe","file":{"name":"a6c7a86b5af85a0de1685c0bcce4b3d","type":"zip","contentDRI":"zQmNiqMUz1tnUsXroHb2ugKoPDGoZCAqrHRz7vYHEHAw3jA"}},{"DRI":"zQmcXWdhiJSaWg25mXKPTZAp4EAt7SjfEPAjKdjSDUXfDXn","file":{"name":"test_file","type":null,"contentDRI":"zQmVAxg5Ncz6C8zjx4bab7uQB2W8AGqkAVcRAi2VceQN6Cc"}},{"DRI":"zQmRBXgMkjh1Vw9MB3XZt3cHDRxmX15b7xmHLsVFcjZv8th","file":{"name":"abba","type":null,"contentDRI":"zQmNmDadhKMDtF1C8nJBwxzync5RytDBJruo9rQ2mgKnQMj"}},{"DRI":"zQmNxuZvDfqHopr3thcQurYePoB11DaEb6wu3qPHpwxEG3u","file":{"name":"string","type":null,"contentDRI":"zQmSGkFU26J43H5ToS5VwoyS1eqP41F6VU1WUyNaNui4gXg"}}]
[{'DRI': 'zQmP9LkvbGXK2aaVuitR8WCZQYeKosFvcFQQjunZkz8NZ4F',
  'file': {'contentDRI': 'zQmbB6LSS92nZEmEKZLQH6bzd9EeBXuW42DuWpSmD76aMQX',
  